# TEAM DANCER PROJECT PART 1
### Joanne Li, Jasmine Kwok, Ryan Logue, Jay Louissaint 

## Part 1: Designing the database

In Part 1 of the project, your team must design a table schema for the data. By “table schema” I mean the CREATE TABLE statements necessary to create database tables that fit the data. You should follow the design principles in Section 4.1 to build a normalized structure for the database that minimizes redundant information. Include primary keys, foreign keys, column types, and any appropriate constraints. It is up to you to decide how many tables you need, their names, and their contents.

Write your CREATE TABLE statements in a notebook. Test them out on Azure to ensure they work correctly. You do not need to load any real data into the database yet.

In the notebook, write comments explaining the following: What are the basic entities in your schema? (In Example 4.1, entities were things like songs, record labels, and albums, that each had their own database table.) How did you choose them and what did you do to ensure there is not redundant information in your database?

From the IPEDS data, obtain:

* All information about the institution’s name, location, address, and similar
* All Carnegie Classification 2021 variables
* The Census identifiers that apply to it: Core Based Statistical Area (CBSA) and its type, the Combined Statistical Area (CSA), and the county FIPS code -- where is this info? 
* Latitude and longitude of the institution.

Most of these values will not change from year to year, but they can, since colleges can change names, move, grow, or change.

The College Scorecard data also includes crosswalk files giving links between OPEIDs (in the College Scorecard data) and UNITIDs (in the IPEDS) data, so you can match between datasets.

Links to Data Dictionary:
* https://collegescorecard.ed.gov/assets/InstitutionDataDocumentation.pdf
* https://collegescorecard.ed.gov/assets/FieldOfStudyDataDocumentation.pdf



## Database Schema Design Reasoning
For this project, the database schema is designed to track institutions and their yearly data while minimizing redundancy and ensuring data integrity. The core principle is to separate attributes that remain stable over time from those that change annually.

### Core Entities:
* Institutions – Stores static information about each university, including name, address, geographic coordinates, accrediting agency, ownership type, and classification codes (e.g., Carnegie classification). Each institution is uniquely identified by a UNITID primary key. These attributes are unlikely to change frequently, so they are stored once per institution.
* Financials, Academics, and Demographics – Store time-dependent information, such as tuition, program offerings, enrollment, and student demographics. Each row corresponds to one institution in a given year. A YEAR column tracks the year of the data, and the UNITID foreign key links each record back to the Institutions table.

### Reference Tables:
We created eight reference tables for categorical columns: PREDDEG, HIGHDEG, CONTROL, REGION, CCBASIC, CCUGPROF, CCSIZSET, and ST_FIPS. These tables map integer IDs to descriptive text values. By storing categories in reference tables and joining them by ID:
We follow the DRY principle, eliminating duplication of repeated text data. We reduce memory usage and computing power, since integers are much more efficient than storing long text strings repeatedly.

### Design Principles:
* Normalization (DRY): Separating static, time-dependent, and categorical data avoids redundancy and ensures updates are centralized.
* Referential Integrity: Foreign keys link yearly data and categorical reference tables to institutions, preventing orphaned records.
* Versioning: Including a YEAR column in time-dependent tables allows tracking historical changes and maintaining a clear record of each dataset version.
* Efficient Storage: Joining with reference tables for categorical data improves performance and reduces storage costs.
* Ease of Analysis & Visualization: Separating data into logical categories (Financials, Academics, Demographics) allows for more flexible filtering and targeted visualizations. For example, we can analyze only demographic factors without loading academic or financial data.

### Summary:
This schema allows efficient storage and querying of both stable and changing attributes of institutions while maintaining consistency, minimizing duplication, and supporting historical, categorical, and visualization-focused analysis.


In [ ]:
%%sql
--- Institutions Table -- 1 row is an institution
/* This table is used to store basic institution information that generally does not change year-to-year. 
Each row represents one institution.
*/

CREATE TABLE Institutions(
    UNITID SERIAL PRIMARY KEY, 
    INSTNM VARCHAR(255) UNIQUE NOT NULL, 
    ADDR VARCHAR(255) NOT NULL,
    CITY VARCHAR(100) NOT NULL,
    STABBR TEXT NOT NULL,
    ZIP VARCHAR(10) NOT NULL,  
    LATITUDE NUMERIC(10, 7), 
    LONGITUDE NUMERIC(10, 7),
    PREDDEG INTEGER NOT NULL CHECK (PREDDEG BETWEEN 0 AND 4)I, 
    HIGHDEG INTEGER NOT NULL CHECK (HIGHDEG BETWEEN 0 AND 4), 
    CONTROL INTEGER NOT NULL CHECK (CONTROL BETWEEN 1 AND 3), 
    REGION INTEGER NOT NULL CHECK (REGION BETWEEN 0 AND 43), 
    CCBASIC INTEGER NOT NULL CHECK (CCBASIC BETWEEN -3 AND 33), 
    CCUGPROF INTEGER NOT NULL CHECK (CCUGPROF BETWEEN -2 AND 15), 
    CCUGSIZSET INTEGER NOT NULL,
    ST_FIPS INTEGER NOT NULL CHECK (ST_FIPS BETWEEN 1 AND 78),
    CountyFIPS VARCHAR(5),
    CSA INTEGER VARCHAR(3),
    CBSA INTEGER VARCHAR(5)
);

--- Institution Financials Table -- Tuition Fees, Salaries, etc 
/* This table is used to store university financial information such as tuition fees, faculty salaries, and so on. 
This information generally changes year-by-year, therefore it includes a YEAR column. 
The INSTID column is a foreign key used to map information to their respective institution in the Institution table.
*/

CREATE TABLE Financials(
    FINID SERIAL PRIMARY KEY, 
    INSTID INTEGER REFERENCES Institutions(UNITID),  
    YEAR INTEGER CHECK (YEAR <= EXTRACT(YEAR FROM CURRENT_DATE), 
    TUITIONFEE_IN INTEGER NOT NUL CHECK(TUITIONFEE_IN >= 0), 
    TUITIONFEE_OUT INTEGER NOT NULL CHECK(TUITIONFEE_OUT >= 0), 
    TUITIONFEE_PROG INTEGER NOT NULL CHECK(TUITIONFEE_PROG >= 0), 
    TUITFTE INTEGER NOT NULL CHECK(TUITFTE >= 0), 
    AVGFASCAL INTEGER CHECK(AVGFASCAL > 0), 
    CDR2 FLOAT CHECK(CDR2 > 0),  
    CDR3 FLOAT CHECK(CDR3 > 0),
    UNIQUE (UNITID, YEAR)
);

--- Institution Academics Table --  Student Outcomes, Graduation Rates, PostGraduation Counts Etc.
/*This table is used to store university academic information such as admission rate, graduation rate, and so on. 
This information also changes year-by-year, therefore it includes a YEAR column. 
The INSTID column is a foreign key used to map information to their respective institution in the Institution table.
*/

CREATE TABLE Academics(
    ACADID SERIAL PRIMARY KEY, 
    UNITID INTEGER REFERENCES Institutions(UNITID), 
    YEAR INTEGER CHECK (YEAR <= EXTRACT(YEAR FROM CURRENT_DATE), 
    ADM_RATE FLOAT CHECK(ADM_RATE BETWEEN 0 AND 1), 
    C100_4 FLOAT CHECK(CDR3 BETWEEN 0 AND 1),
    C100_L4 FLOAT CHECK(CDR3 BETWEEN 0 AND 1),
    SAT_AVG FLOAT CHECK(SAT_AVG >= 0),
    COUNT_NWNE_3YR INTEGER CHECK(COUNT_NWNE_3YR >= 0), 
    COUNT_NWNE_3YR INTEGER CHECK(COUNT_NWNE_3YR >= 0), 
    CNTOVER150_3YR INTEGER CHECK(COUNT_NWNE_3YR >= 0)
    UNIQUE (UNITID, YEAR)
);

-- Demographics Table -- Demographic, Faculty Info
/* This table is used to store university demographic information such as the percentage of different ethnicities. 
This information also changes year-by-year, therefore it includes a YEAR column. 
The INSTID column is a foreign key used to map information to their respective institution in the Institution table.
*/

CREATE TABLE Demographics(
    DEMOID SERIAL PRIMARY KEY, 
    INSTID INTEGER REFERENCES Institutions(UNITID),  
    YEAR INTEGER CHECK (YEAR <= EXTRACT(YEAR FROM CURRENT_DATE), 
    UGDS INTEGER CHECK(ADM_RATE >= 0),  
    UGDS_MEN FLOAT CHECK(UGDS_MEN BETWEEN 0 AND 1),  
    UGDS_WOMEN FLOAT CHECK(UGDS_WOMEN BETWEEN 0 AND 1),
    UGDS_WHITE FLOAT CHECK(UGDS_WHITE BETWEEN 0 AND 1),  
    UGDS_BLACK FLOAT CHECK(UGDS_BLACK BETWEEN 0 AND 1),
    UGDS_HISP FLOAT CHECK(UGDS_HISP BETWEEN 0 AND 1),  
    UGDS_ASIAN FLOAT CHECK(UGDS_ASIAN BETWEEN 0 AND 1),
    UGDS_AIAN FLOAT CHECK(UGDS_AIAN BETWEEN 0 AND 1),  
    UGDS_NHPI FLOAT CHECK(UGDS_NHPI BETWEEN 0 AND 1),
    UGDS_2MOR FLOAT CHECK(UGDS_2MOR BETWEEN 0 AND 1),  
    UGDS_UNKN FLOAT CHECK(UGDS_UNKN BETWEEN 0 AND 1),
    IRPS_MEN FLOAT CHECK(IRPS_MEN BETWEEN 0 AND 1),  
    IRPS_WOMEN FLOAT CHECK(IRPS_WOMEN BETWEEN 0 AND 1),
    IRPS_WHITE FLOAT CHECK(IRPS_WHITE BETWEEN 0 AND 1),  
    IRPS_BLACK FLOAT CHECK(IRPS_BLACK BETWEEN 0 AND 1),
    IRPS_HISP FLOAT CHECK(IRPS_HISP BETWEEN 0 AND 1),  
    IRPS_ASIAN FLOAT CHECK(IRPS_ASIAN BETWEEN 0 AND 1),
    IRPS_AIAN FLOAT CHECK(IRPS_AIAN BETWEEN 0 AND 1),  
    IRPS_NHPI FLOAT CHECK(IRPS_NHPI BETWEEN 0 AND 1),
    IRPS_2MOR FLOAT CHECK(IRPS_2MOR BETWEEN 0 AND 1),  
    IRPS_UNKN FLOAT CHECK(IRPS_UNKN BETWEEN 0 AND 1),
    UNIQUE (INSTID, YEAR)
    CHECK (SUM(UDGS_WHITE, UGDS_BLACK, UGDS_HISP, UGDS_ASIAN, UGDS_AIAN, UGDS_NHPI, UGDS_2MOR, UGDS_UNKN) = 1)
    CHECK (SUM(IRPS_WHITE, IRPS_BLACK, IRPS_HISP, IRPS_ASIAN, IRPS_AIAN, IRPS_NHPI, IRPS_2MOR, IRPS_UNKN) = 1)
);

In [ ]:
%%sql
-- Reference Tables 
-- Predominant Undergraduate Degree Awarded
CREATE TABLE preddeg_reference (
    code INTEGER PRIMARY KEY,
    value TEXT NOT NULL,
    description TEXT
);


-- Highest Degree Awarded
CREATE TABLE highdeg_reference (
    code INTEGER PRIMARY KEY,
    value TEXT NOT NULL,
    description TEXT
);


-- Control of Institution
CREATE TABLE control_reference (
    code INTEGER PRIMARY KEY,
    value TEXT NOT NULL,
    description TEXT
);


-- Region and Locale Classification
CREATE TABLE region_reference (
    code INTEGER PRIMARY KEY,
    value TEXT NOT NULL,
    description TEXT
);


-- Carnegie Basic Classification
CREATE TABLE ccbasic_reference (
    code INTEGER PRIMARY KEY,
    value TEXT NOT NULL,
    description TEXT
);


-- Undergraduate Profile
CREATE TABLE ccugprof_reference (
    code INTEGER PRIMARY KEY,
    value TEXT NOT NULL,
    description TEXT
);


-- State FIPS (ST_FIPS)
CREATE TABLE st_fips_reference (
    code INTEGER PRIMARY KEY,
    state_name TEXT NOT NULL,
    description TEXT
);

-- Size and Setting Classification (CCSIZSET) and 